In [2]:
# find files recursively
from glob import glob

# parse tb files
from tbparse import SummaryReader

# plotting
import matplotlib.pyplot as plt
import seaborn as sns

# searching df rows with np.where()
import numpy as np
import pandas as pd

import re

2024-09-03 13:22:28.571433: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [4]:
# recursively read tensorboard files
tb_files = []
# for filename in glob('**/events.out.tfevents.*', recursive=True):
#     tb_files.append(filename)
for filename in glob('training/spiderenv_training/**/*reward_sum*', recursive=True):
    tb_files.append(filename)
for filename in glob('**/seqdqn_***', recursive = True):
    tb_files.append(filename)
for filename in glob('**/dqn_**', recursive = True):
    tb_files.append(filename)
for filename in glob('**/seqsac_**', recursive = True):
    tb_files.append(filename)
for filename in glob('**/sac_**', recursive = True):
    tb_files.append(filename)

# tb_files = [file for file in tb_files if not "eudist" in file and not "seqdqn_3_spider_2" in file and not "lowlr2" in file]
tb_files = [file for file in tb_files if "meeting" in file and not "meeting_training_old" in file and not "spread_training" in file and not "spider" in file and not "custom_agents" in file]
# tb_files = [file for file in tb_files if not "eudist" in file and not "seqdqn_3_spider_2" in file]
# print([file for file in tb_files if not "eudist" in file])
tb_files = [file for file in tb_files if "/dqn_3" in file and not "/dqn_3_25" in file]

# tb_files = [re.sub("training/spiderenv_training/", "", re.sub("_\d(/tensorboard_logs/rollout_reward_sum_agent\d)?", "", file)) for file in tb_files if not "eudist" in file]
tb_files

['training/meeting_training/dqn_3_75',
 'training/meeting_training/dqn_3_hlr_25',
 'training/meeting_training/dqn_3_hlr_10',
 'training/meeting_training/dqn_3_llr_10',
 'training/meeting_training/dqn_3_10',
 'training/meeting_training/dqn_3_llr_25',
 'training/meeting_training/dqn_3_hlr_50',
 'training/meeting_training/dqn_3_50',
 'training/meeting_training/dqn_3_llr_50',
 'training/meeting_training/dqn_3_llr_75',
 'training/meeting_training/dqn_3_hlr_75']

In [5]:
df = []
for file in tb_files:
    reader = SummaryReader(file, pivot = True)
    temp_df = reader.scalars
    if "meeting_training/sac_" in reader.log_path:
        temp_df.rename(columns={"reward/reward_sum":"rollout/reward_sum"}, inplace=True)
    print(reader.log_path)
    if "rollout/reward_sum" in temp_df.columns:
        temp_df = temp_df[["step", "rollout/reward_sum"]] 
        print(re.sub("training/meeting_training/", "", re.sub("(_)?(\d)?(/tensorboard_logs/rollout_reward_sum_agent\d)|((_)?(\d)?)$", "",  reader.log_path)))
        temp_df.insert(2, "dir_name", np.repeat(re.sub("training/meeting_training/", "", re.sub("(_)?(\d)?(/tensorboard_logs/rollout_reward_sum_agent\d)|((_)?(\d)?)$", "",  reader.log_path)), len(reader.scalars.index)))
        # temp_df.insert(2, "dir_name", np.repeat(reader.log_path.replace("/tensorboard_logs/rollout_reward_sum_agent*", ""), len(reader.scalars.index)))
        temp_df["step"] = temp_df.index

        if "seqsac" in reader.log_path or "seqdqn" in reader.log_path:
            temp_df['rollout/reward_sum'] = temp_df['rollout/reward_sum'].apply(np.mean)
        temp_df["avg_reward_sum"] = temp_df["rollout/reward_sum"].rolling(window = 25, step = 25).mean()
        temp_df = temp_df[temp_df.step > 25]
        temp_df = temp_df[temp_df['avg_reward_sum'].notna()]
        # temp_df = temp_df[["step", "rollout/reward_sum", "dir_name", "avg_reward_sum"]]
        temp_df.dropna()
        df.append(temp_df)
        print("succes")
    else:
        print("reward logs not found")
print("all success")

df = pd.concat(df)
df = df[df['avg_reward_sum'].notna()]

plt.figure(figsize=(10,6))
sns.lineplot(data = df, x = df.index, y = "avg_reward_sum", hue = "dir_name")
# sns.lineplot(data = df, x = df.index, y = "rollout/reward_sum", hue = "dir_name")
# plt.legend([],[], frameon = False)
# sns.lineplot(data = df, x = "step", y = "rollout/reward_sum", hue = "dir_name")
# sns.lineplot(data = df[df["dir_name"].str.contains("custom")], x = "step", y = "rollout/reward_sum", hue = "dir_name")
plt.legend(bbox_to_anchor = (0, 1), loc = 'center right', ncol = 1)

training/meeting_training/dqn_3_75
dqn_3_7
succes
training/meeting_training/dqn_3_hlr_25
dqn_3_hlr_2
succes
training/meeting_training/dqn_3_hlr_10
dqn_3_hlr_1
succes


: 